In [ ]:
import sys
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import urllib.request
from konlpy.tag import Okt
import socket
import _thread
from pydub import AudioSegment
import os
import speech_recognition as sr
import json
import pickle

file_name = "LSTM_module_ver3.0"

# GPU 있는 경우 활성화
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

with open(file_name+'_tokenizer.pickle', 'rb') as f:
    tokenizer = pickle.load(f)
    
with open(file_name+'_stopwords.json') as f:
    stopwords = json.load(f)

with open(file_name+'_variable.json') as f:
    var = json.load(f)
    
max_data = var['max_data'] # 데이터 문자열 최대 길이
min_data = var['min_data'] # 데이터 문자열 최소 길이
max_len = var['max_len'] # 전체 데이터 셋 길이 설정 (메일의 최대 길이)

model = tf.keras.models.load_model(file_name+'.h5') # 모델 로드
model.summary()

okt = Okt()

def preTreatment(data):
    global stopwords
    temp = okt.morphs(data, stem=True) # 토큰화
    temp = [word for word in temp if not word in stopwords] # 불용어 처리
    return temp


def sentiment_predict(new_sentence):
    data = tokenizer.texts_to_sequences([new_sentence]) # 단어를 숫자값, 인덱스로 변환하여 저장
    pad_new = pad_sequences(data, maxlen = max_len) # 모든 메일의 길이를 100로 설정 (빈 부분은 0으로 패딩)
    score = float(model.predict(pad_new))
    print("[{:.2f}% 확률로 스팸입니다.]".format(score * 100))
    print("-------------------------------------------------------------------")
    return score

def STTtransform(file_name):
    r = sr.Recognizer()
    harvard = sr.AudioFile(file_name) # 100MB 용량 제한
    with harvard as source:
        audio = r.record(source)

    t = r.recognize_google(audio, language='ko-KR')

    temp_X = []
    x = [t[i:i+max_data] for i in range(0, len(t), max_data)] # 텍스트 파일 중 150 길이로 데이터 길이 제한

    for s in x:
        if len(s) > min_data: # 길이가 50 이하인 데이터 삭제
            temp_X.append(s)
    x = temp_X
    score_sum = 0.0
    if len(x) == 0:
        return 0.0 # 음성 길이가 너무 짧을 때
    
    print('-------------------------------------------------------------------')
    for txt in x:
        print(txt)
        data = preTreatment(txt)
        score_sum += sentiment_predict(data)
    score_result = score_sum / len(x)
    return score_result

def formatTransform(file_name):
    recording = AudioSegment.from_file(file_name)
    recording = recording + 6 # 볼륨 향상
    formatFlac = file_name.replace(".m4a",".flac")
    recording.export(formatFlac, format='flac', parameters=["-q:a", "10", "-ac", "1"])
    print("Convert : " + file_name + " -> " + formatFlac)
    os.remove(file_name) # 변환 전 파일 제거
    print("remove : " + file_name)
    return formatFlac

def threaded(conn, addr, group):
    print('Connect by',addr)
    while True:
        try:
            data = conn.recv(1024)
            if not data:
                print('Disconnected by',addr)
                group.remove(conn)
                break
                
            file_info = data.decode()
            file_name, file_size = file_info.split("/")
            print('Receive File Path:',file_name)
            print('Receive File Size:',file_size)

            data = conn.recv(1024)
            data_transferred = len(data)
            with open(file_name, "wb") as f:
                try:
                    while data:
                        f.write(data)
                        data = conn.recv(1024)
                        data_transferred += len(data)
                        
                        if data_transferred == int(file_size): # 파일 다 받으면 break
                            f.write(data)
                            break
                except Exception as ex:
                    print(ex)

            print("File is saved [byte:"+str(data_transferred)+"]")
            
            formatFlac = formatTransform(file_name)
            score = STTtransform(formatFlac)
            
            if score == 0.0:
                txt = "녹음 길이가 너무 짧습니다.\n"
            else:
                txt = "{:.2f}% 확률로 스팸입니다.\n".format(score * 100) # Client측 readLine 함수로 인한 개행문자(\n)필요
            
            sendData = txt
            for c in group:
                if c is conn:
                    c.sendall(bytes(sendData,'UTF-8')) # 수신된 파일을 보낸 Client에게만 전송
                    print('Send Data : '+txt,end="")
                    os.remove(formatFlac) # 변환 후 파일 제거
                    print("remove : " + formatFlac)
        
        except:
            # 클라이언트 소켓 강제 종료 시 (ex : 네트워크 변경)
            print('예외발생')
            print('Disconnected by',addr)
            group.remove(conn)
            break
    conn.close()

host = ''
port = 50000

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_socket.bind((host, port))
server_socket.listen()
print("Listening")

group = []
while True:
    conn, addr = server_socket.accept()
    group.append(conn)
    _thread.start_new_thread(threaded, (conn, addr, group))
    
server_socket.close()
print('서버 종료')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 60)          624840    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               64400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 689,341
Trainable params: 689,341
Non-trainable params: 0
_________________________________________________________________
Listening
Connect by ('192.168.0.37', 65373)
Receive File Path: 211012_133618
Receive File Size: 0
File is saved [byte:0]
예외발생
Disconnected by ('192.168.0.37', 65373)
Connect by ('192.168.0.37', 65389)
Receive File Path: 통화 녹음 인턴 김수빈_211012_133618.m4a
Receive File Size: 1024443
File is saved [byte:1024443]
Convert : 통화 녹음 인턴 김수빈_211012_133618.m4a -> 통화 녹음 인턴 김수빈_211012_133618.f